In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from numba import jit, prange, njit
from blimpy import Waterfall
import time
import random
import datetime
from synthetic_real import create_true, create_full_cadence, create_false, create_true_single_shot, create_true_faster
from scipy import spatial
from numpy import dot
from numpy.linalg import norm
import math
from sklearn.metrics import silhouette_score
import sys
sys.path.insert(1, '../ML_Training')
sys.path.insert(2, '../GBT_pipeline')
from preprocess import get_data
from single_search import search_model_eval, combine

@jit(nopython=True)
def pre_proc(data):
    data = np.log(data)
    data= data - data.min()
    data = data/data.max()
    return data

@jit(parallel=True)
def load_data_ED(data):
    print(data.shape)
    data_transform =  np.zeros((data.shape[0],6, 16,256,1))
    for i in prange(data.shape[0]):
        data_transform[i,:,:,:,0]  = pre_proc(data[i,:,:,:] )
    return data_transform

def combine(data):
    new_data = np.zeros((data.shape[0]*data.shape[1],data.shape[2],data.shape[3],data.shape[4]))
    for i in prange(data.shape[0]):
        new_data[i*data.shape[1] : (i+1)*data.shape[1],:,:,:] = data[i,:,:,:,:]
    return new_data
# tf.config.run_functions_eagerly(False)

/home/pma/.local/lib/python3.6/site-packages/numba/core/decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
%matplotlib inline
NUM_SAMPLES = 20000
plate = np.load('../../real_filtered.npy')[:77687]

print(plate.shape)
print("Creating True")
data = create_full_cadence(create_true_faster, plate = plate, samples = NUM_SAMPLES,  snr_base=10, snr_range=30, factor =0.1)
data = combine(load_data_ED(data))

print("Creating False")
false_data = abs(create_full_cadence(create_false, plate = plate, samples = NUM_SAMPLES*6, snr_base=10, snr_range=30))
false_data = load_data_ED(false_data)

print("Creating True")
true_data = create_full_cadence(create_true_faster, plate = plate, samples = NUM_SAMPLES*6,  snr_base=10, snr_range=30, factor =0.1)
true_data = load_data_ED(true_data)

(77687, 6, 16, 256)
Creating True
(20000, 6, 16, 256)
Creating False
(120000, 6, 16, 256)
Creating True
(120000, 6, 16, 256)


In [3]:
NUM_SAMPLES = 1000

print(plate.shape)
print("Creating True")
data_test = create_full_cadence(create_true_faster, plate = plate, samples = NUM_SAMPLES,  snr_base=10, snr_range=30, factor =0.1)
data_test = combine(load_data_ED(data_test))

print("Creating False")
false_data_test = abs(create_full_cadence(create_false, plate = plate, samples = NUM_SAMPLES*6, snr_base=10, snr_range=30))
false_data_test = load_data_ED(false_data_test)


print("Creating True")
true_data_test  = create_full_cadence(create_true_faster, plate = plate, samples = NUM_SAMPLES*6,  snr_base=10, snr_range=30, factor =0.1)
true_data_test = load_data_ED(true_data_test)

(77687, 6, 16, 256)
Creating True
(1000, 6, 16, 256)
Creating False
(6000, 6, 16, 256)
Creating True
(6000, 6, 16, 256)


In [13]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder,  **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.true_loss_tracker = keras.metrics.Mean(name="true_loss")
        self.false_loss_tracker = keras.metrics.Mean(name="false_loss")
        
        self.total_loss_tracker_validation = keras.metrics.Mean(name="val_total_loss")
        self.reconstruction_loss_tracker_validation = keras.metrics.Mean(
            name="val_reconstruction_loss"
        )
        self.kl_loss_tracker_validation = keras.metrics.Mean(name="val_kl_loss")
        self.false_loss_tracker_validation = keras.metrics.Mean(name="val_false_loss")
        self.true_loss_tracker_validation = keras.metrics.Mean(name="val_true_loss")
        
        alpha=2
        beta=2
        self.alpha = alpha
        self.beta = beta
        self.count=1

    @tf.function
    def loss_diff(self, a,b):
        return 1/ self.loss_same(a,b)
   
    @tf.function
    def loss_same(self, a,b):
        return tf.math.reduce_mean(tf.math.reduce_euclidean_norm(a-b, axis=1))

    
    @tf.function
    def true_clustering(self, true_data,training=True):
        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(true_data[:,0,:,:,:], training)[2]
        b = self.encoder(true_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(true_data[:,2,:,:,:],training)[2]
        c = self.encoder(true_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(true_data[:,4,:,:,:], training)[2]
        d = self.encoder(true_data[:,5,:,:,:], training)[2]

        difference += self.loss_diff(a1,b)
        difference += self.loss_diff(a1,c)
        difference += self.loss_diff(a1,d)

        difference += self.loss_diff(a2,b)
        difference += self.loss_diff(a2,c)
        difference += self.loss_diff(a2,d)

        difference += self.loss_diff(a3,b)
        difference += self.loss_diff(a3,c)
        difference += self.loss_diff(a3,d)

        same += self.loss_same(a1,a2)
        same += self.loss_same(a1,a3)
        same += self.loss_same(a2,a3)
        
        same += self.loss_same(b,c)
        same += self.loss_same(c,d)
        same += self.loss_same(b,d)
        

        similarity += same+difference
        return similarity
    
    @tf.function
    def false_clustering(self, false_data, training=True):

        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(false_data[:,0,:,:,:], training)[2]
        b = self.encoder(false_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(false_data[:,2,:,:,:],training)[2]
        c = self.encoder(false_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(false_data[:,4,:,:,:], training)[2]
        d = self.encoder(false_data[:,5,:,:,:], training)[2]

        difference += self.loss_same(a1,b)
        difference += self.loss_same(a1,c)
        difference += self.loss_same(a1,d)

        difference += self.loss_same(a2,b)
        difference += self.loss_same(a2,c)
        difference += self.loss_same(a2,d)

        difference += self.loss_same(a3,b)
        difference += self.loss_same(a3,c)
        difference += self.loss_same(a3,d)

        same += self.loss_same(a1,a2)
        same += self.loss_same(a1,a3)
        same += self.loss_same(a2,a3)
        
        same += self.loss_same(b,c)
        same += self.loss_same(c,d)
        same += self.loss_same(b,d)
        
        similarity += same+difference
        return similarity
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        self.count+=1
        cluster_loss =0
        x, y = data
        true_data = x[1]
        false_data = x[2]
        x= x[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(y, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            false_loss = self.false_clustering(false_data)
            true_loss = self.true_clustering(true_data)
            total_loss = reconstruction_loss + self.beta*kl_loss +self.alpha*(30*true_loss+false_loss)
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.false_loss_tracker.update_state(false_loss)
        self.true_loss_tracker.update_state(true_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "false_loss": self.false_loss_tracker.result(),
            "true_loss": self.true_loss_tracker.result()
        }
    

    def test_step(self, data):
        # Unpack the data
        x, y = data
        true_data = x[1]
        false_data = x[2]
        x= x[0]
        z_mean, z_log_var, z = self.encoder(x, training=False)
        reconstruction = self.decoder(z, training=False)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(y, reconstruction), axis=(1, 2)
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        false_loss = self.false_clustering(false_data, False)
        true_loss = self.true_clustering(true_data, False)
        total_loss = reconstruction_loss + self.beta*kl_loss +self.alpha*(30*true_loss+false_loss)
        
        
        self.total_loss_tracker_validation.update_state(total_loss)
        self.reconstruction_loss_tracker_validation.update_state(reconstruction_loss)
        self.kl_loss_tracker_validation.update_state(kl_loss)
        self.false_loss_tracker_validation.update_state(false_loss)
        self.true_loss_tracker_validation.update_state(true_loss)
        
        return {
            "loss": self.total_loss_tracker_validation.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker_validation.result(),
            "kl_loss": self.kl_loss_tracker_validation.result(),
            "false_loss": self.false_loss_tracker_validation.result(),
            "true_loss": self.true_loss_tracker_validation.result()
        }

In [14]:
from tensorflow.python.client import device_lib
gpus = tf.config.experimental.list_physical_devices('GPU')

In [15]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [16]:
def sample_creation(inputs):
    z_mean = inputs[0]
    z_log_var = inputs[1]
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [17]:
def build_model():
    latent_dim = 8
    dens_lay = 512
    kernel = (3,3)
    encoder_inputs = keras.Input(shape=(16, 256, 1))
    x = layers.Conv2D(16, kernel, activation="relu", strides=2, padding="same")(encoder_inputs)
    x = layers.Conv2D(16, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(64, kernel, activation="relu", strides=2,padding="same")(x)
    x = layers.Conv2D(64, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(128, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(256, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(dens_lay, activation="relu")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    encoder.summary()
    
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(dens_lay, activation="relu")(latent_inputs)
    x = layers.Dense(1* 16 * 256, activation="relu")(x)
    x = layers.Reshape((1,16, 256))(x)
    x = layers.Conv2DTranspose(256, kernel, activation="relu", strides=2,padding="same")(x)
    x = layers.Conv2DTranspose(128, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(64, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(64, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(16, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(16, kernel, activation="relu", strides=2, padding="same")(x)
    decoder_outputs = layers.Conv2DTranspose(1, kernel, activation="sigmoid", padding="same")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    decoder.summary()

    vae = VAE(encoder, decoder)
    vae.compile(optimizer=keras.optimizers.Adam(lr=0.0005))
    return vae

strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice())
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = build_model()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
tensorflow      INFO     Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 16, 256, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 8, 128, 16)   160         in

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="vae_best_model.hdf5",
    monitor='val_loss',
    save_weights_only=False,
    mode='min',
    save_best_only=True)

history = model.fit(x = [data,true_data,false_data ],y= data[:,:,:,:], epochs=800, batch_size=3000, 
          validation_data=([data_test,true_data_test,false_data_test ], data_test),validation_batch_size=6000)

Epoch 1/800
40/40 [==============================] - 78s 1s/step - loss: 4059.9436 - reconstruction_loss: 2786.6987 - kl_loss: 9.3861 - false_loss: 33.6449 - true_loss: 19.3563 - val_loss: 3649.9421 - val_reconstruction_loss: 2716.1973 - val_kl_loss: 7.3410 - val_false_loss: 13.9165 - val_true_loss: 14.8538
Epoch 2/800
40/40 [==============================] - 40s 989ms/step - loss: 3630.3243 - reconstruction_loss: 2684.0256 - kl_loss: 7.9643 - false_loss: 20.4646 - true_loss: 15.9894 - val_loss: 3621.8467 - val_reconstruction_loss: 2687.5835 - val_kl_loss: 8.1982 - val_false_loss: 14.3277 - val_true_loss: 14.8202
Epoch 3/800
40/40 [==============================] - 40s 996ms/step - loss: 3592.6524 - reconstruction_loss: 2658.5430 - kl_loss: 9.2321 - false_loss: 18.1442 - true_loss: 15.4858 - val_loss: 3610.1758 - val_reconstruction_loss: 2677.0828 - val_kl_loss: 8.8427 - val_false_loss: 14.2732 - val_true_loss: 14.7810
Epoch 4/800
40/40 [==============================] - 41s 1s/step - 

40/40 [==============================] - 41s 1s/step - loss: 3154.3958 - reconstruction_loss: 2755.2151 - kl_loss: 70.7733 - false_loss: 17.2716 - true_loss: 5.6906 - val_loss: 3228.6641 - val_reconstruction_loss: 2746.9485 - val_kl_loss: 64.3716 - val_false_loss: 15.0610 - val_true_loss: 5.3808
Epoch 28/800
40/40 [==============================] - 41s 1s/step - loss: 3149.2981 - reconstruction_loss: 2755.1265 - kl_loss: 70.7381 - false_loss: 17.2707 - true_loss: 5.6167 - val_loss: 3226.0620 - val_reconstruction_loss: 2747.2583 - val_kl_loss: 64.6955 - val_false_loss: 15.0645 - val_true_loss: 5.3214
Epoch 29/800
40/40 [==============================] - 41s 1s/step - loss: 3151.3901 - reconstruction_loss: 2755.1277 - kl_loss: 70.7469 - false_loss: 17.2759 - true_loss: 5.5484 - val_loss: 3223.6423 - val_reconstruction_loss: 2747.5469 - val_kl_loss: 64.9456 - val_false_loss: 15.0545 - val_true_loss: 5.2683
Epoch 30/800
40/40 [==============================] - 41s 1s/step - loss: 3146.3500

40/40 [==============================] - 41s 1s/step - loss: 3151.3704 - reconstruction_loss: 2755.0305 - kl_loss: 70.7826 - false_loss: 17.3578 - true_loss: 4.6916 - val_loss: 3193.0200 - val_reconstruction_loss: 2751.1733 - val_kl_loss: 69.6277 - val_false_loss: 15.2798 - val_true_loss: 4.5339
Epoch 54/800
40/40 [==============================] - 41s 1s/step - loss: 3149.3506 - reconstruction_loss: 2754.7598 - kl_loss: 70.1412 - false_loss: 17.3590 - true_loss: 4.6719 - val_loss: 3192.2998 - val_reconstruction_loss: 2751.2581 - val_kl_loss: 69.6831 - val_false_loss: 15.2740 - val_true_loss: 4.5188
Epoch 55/800
40/40 [==============================] - 41s 1s/step - loss: 3150.2012 - reconstruction_loss: 2754.8228 - kl_loss: 70.8900 - false_loss: 17.3617 - true_loss: 4.6536 - val_loss: 3191.6858 - val_reconstruction_loss: 2751.3396 - val_kl_loss: 69.6860 - val_false_loss: 15.2635 - val_true_loss: 4.5075
Epoch 56/800
40/40 [==============================] - 41s 1s/step - loss: 3154.7742

40/40 [==============================] - 41s 1s/step - loss: 3102.7110 - reconstruction_loss: 2753.9348 - kl_loss: 61.1495 - false_loss: 17.8492 - true_loss: 4.2814 - val_loss: 3173.2629 - val_reconstruction_loss: 2752.4727 - val_kl_loss: 69.5396 - val_false_loss: 15.7610 - val_true_loss: 4.1698
Epoch 80/800
40/40 [==============================] - 41s 1s/step - loss: 3096.3358 - reconstruction_loss: 2754.2349 - kl_loss: 60.9736 - false_loss: 17.8850 - true_loss: 4.2660 - val_loss: 3172.4045 - val_reconstruction_loss: 2752.4956 - val_kl_loss: 69.4727 - val_false_loss: 15.7929 - val_true_loss: 4.1563
Epoch 81/800
40/40 [==============================] - 41s 1s/step - loss: 3104.5017 - reconstruction_loss: 2754.2817 - kl_loss: 61.1765 - false_loss: 17.9241 - true_loss: 4.2516 - val_loss: 3171.5676 - val_reconstruction_loss: 2752.5215 - val_kl_loss: 69.3791 - val_false_loss: 15.8156 - val_true_loss: 4.1443
Epoch 82/800
40/40 [==============================] - 41s 1s/step - loss: 3103.1322

40/40 [==============================] - 40s 1s/step - loss: 3147.4841 - reconstruction_loss: 2755.0652 - kl_loss: 70.2543 - false_loss: 18.1600 - true_loss: 4.0628 - val_loss: 3164.8789 - val_reconstruction_loss: 2753.1965 - val_kl_loss: 69.5927 - val_false_loss: 15.9752 - val_true_loss: 4.0091
Epoch 106/800
40/40 [==============================] - 41s 1s/step - loss: 3149.6117 - reconstruction_loss: 2755.0837 - kl_loss: 69.7096 - false_loss: 18.1534 - true_loss: 4.0587 - val_loss: 3164.7551 - val_reconstruction_loss: 2753.2200 - val_kl_loss: 69.6333 - val_false_loss: 15.9716 - val_true_loss: 4.0054
Epoch 107/800
40/40 [==============================] - 40s 1s/step - loss: 3149.8736 - reconstruction_loss: 2755.0815 - kl_loss: 67.5756 - false_loss: 18.1463 - true_loss: 4.0547 - val_loss: 3164.3792 - val_reconstruction_loss: 2753.2424 - val_kl_loss: 69.6125 - val_false_loss: 15.9841 - val_true_loss: 3.9991
Epoch 108/800
40/40 [==============================] - 40s 1s/step - loss: 3118.6

...

40/40 [==============================] - 40s 1s/step - loss: 2967.5726 - reconstruction_loss: 2625.7649 - kl_loss: 60.3856 - false_loss: 19.5712 - true_loss: 3.4724 - val_loss: 3060.8184 - val_reconstruction_loss: 2684.8582 - val_kl_loss: 67.0987 - val_false_loss: 17.5888 - val_true_loss: 3.4431
Epoch 418/800
40/40 [==============================] - 40s 1s/step - loss: 2966.5170 - reconstruction_loss: 2625.3394 - kl_loss: 60.4656 - false_loss: 19.5761 - true_loss: 3.4711 - val_loss: 3060.5728 - val_reconstruction_loss: 2684.7061 - val_kl_loss: 67.0851 - val_false_loss: 17.5944 - val_true_loss: 3.4418
Epoch 419/800
40/40 [==============================] - 40s 1s/step - loss: 2966.7669 - reconstruction_loss: 2626.5815 - kl_loss: 60.1425 - false_loss: 19.5802 - true_loss: 3.4699 - val_loss: 3060.3442 - val_reconstruction_loss: 2684.5698 - val_kl_loss: 67.0695 - val_false_loss: 17.5994 - val_true_loss: 3.4406
Epoch 420/800
40/40 [==============================] - 40s 1s/step - loss: 2968.7

40/40 [==============================] - 41s 1s/step - loss: 2963.3858 - reconstruction_loss: 2620.3608 - kl_loss: 61.3029 - false_loss: 19.6848 - true_loss: 3.4420 - val_loss: 3054.8430 - val_reconstruction_loss: 2681.1392 - val_kl_loss: 66.7543 - val_false_loss: 17.7197 - val_true_loss: 3.4126
Epoch 444/800
40/40 [==============================] - 41s 1s/step - loss: 2961.0021 - reconstruction_loss: 2620.6404 - kl_loss: 60.7497 - false_loss: 19.6887 - true_loss: 3.4409 - val_loss: 3054.6196 - val_reconstruction_loss: 2680.9949 - val_kl_loss: 66.7456 - val_false_loss: 17.7252 - val_true_loss: 3.4114
Epoch 445/800
40/40 [==============================] - 41s 1s/step - loss: 2966.6399 - reconstruction_loss: 2620.7515 - kl_loss: 61.3309 - false_loss: 19.6932 - true_loss: 3.4399 - val_loss: 3054.4097 - val_reconstruction_loss: 2680.8652 - val_kl_loss: 66.7331 - val_false_loss: 17.7299 - val_true_loss: 3.4103
Epoch 446/800
40/40 [==============================] - 41s 1s/step - loss: 2958.6

40/40 [==============================] - 41s 1s/step - loss: 2958.3801 - reconstruction_loss: 2617.7607 - kl_loss: 60.9287 - false_loss: 19.7851 - true_loss: 3.4147 - val_loss: 3049.2859 - val_reconstruction_loss: 2677.5500 - val_kl_loss: 66.5048 - val_false_loss: 17.8460 - val_true_loss: 3.3839
Epoch 470/800
40/40 [==============================] - 41s 1s/step - loss: 2959.8482 - reconstruction_loss: 2618.3406 - kl_loss: 61.1391 - false_loss: 19.7884 - true_loss: 3.4137 - val_loss: 3049.0808 - val_reconstruction_loss: 2677.4165 - val_kl_loss: 66.4970 - val_false_loss: 17.8509 - val_true_loss: 3.3828
Epoch 471/800
40/40 [==============================] - 40s 1s/step - loss: 2958.2200 - reconstruction_loss: 2617.9729 - kl_loss: 61.1755 - false_loss: 19.7923 - true_loss: 3.4127 - val_loss: 3048.8748 - val_reconstruction_loss: 2677.2825 - val_kl_loss: 66.4854 - val_false_loss: 17.8546 - val_true_loss: 3.3819
Epoch 472/800
40/40 [==============================] - 40s 1s/step - loss: 2962.3

40/40 [==============================] - 41s 1s/step - loss: 2963.8191 - reconstruction_loss: 2619.3774 - kl_loss: 60.9302 - false_loss: 19.8750 - true_loss: 3.3899 - val_loss: 3044.2605 - val_reconstruction_loss: 2674.3003 - val_kl_loss: 66.2885 - val_false_loss: 17.9609 - val_true_loss: 3.3577
Epoch 496/800
40/40 [==============================] - 41s 1s/step - loss: 2958.8687 - reconstruction_loss: 2616.5325 - kl_loss: 61.3267 - false_loss: 19.8785 - true_loss: 3.3890 - val_loss: 3044.0725 - val_reconstruction_loss: 2674.1770 - val_kl_loss: 66.2764 - val_false_loss: 17.9636 - val_true_loss: 3.3569
Epoch 497/800
40/40 [==============================] - 41s 1s/step - loss: 2960.0058 - reconstruction_loss: 2616.5164 - kl_loss: 61.2015 - false_loss: 19.8818 - true_loss: 3.3881 - val_loss: 3043.8857 - val_reconstruction_loss: 2674.0554 - val_kl_loss: 66.2681 - val_false_loss: 17.9674 - val_true_loss: 3.3560
Epoch 498/800
40/40 [==============================] - 40s 1s/step - loss: 2958.7

40/40 [==============================] - 40s 1s/step - loss: 2963.3369 - reconstruction_loss: 2619.4536 - kl_loss: 60.7350 - false_loss: 19.9555 - true_loss: 3.3676 - val_loss: 3039.6741 - val_reconstruction_loss: 2671.3059 - val_kl_loss: 66.0769 - val_false_loss: 18.0591 - val_true_loss: 3.3349
Epoch 522/800
40/40 [==============================] - 40s 1s/step - loss: 2956.3918 - reconstruction_loss: 2616.3186 - kl_loss: 60.9487 - false_loss: 19.9584 - true_loss: 3.3668 - val_loss: 3039.5066 - val_reconstruction_loss: 2671.1968 - val_kl_loss: 66.0715 - val_false_loss: 18.0634 - val_true_loss: 3.3340
Epoch 523/800
40/40 [==============================] - 40s 1s/step - loss: 2961.6367 - reconstruction_loss: 2616.9956 - kl_loss: 61.2498 - false_loss: 19.9613 - true_loss: 3.3660 - val_loss: 3039.3372 - val_reconstruction_loss: 2671.0854 - val_kl_loss: 66.0640 - val_false_loss: 18.0669 - val_true_loss: 3.3332
Epoch 524/800
40/40 [==============================] - 40s 1s/step - loss: 2960.6

In [12]:
model.encoder.save("VAE-ENCODERvmini_59_double.h5")
# model.decoder.save("VAE-DECODERv14.h5")

In [ ]:
%matplotlib inline
import random 
for i in range(10):
    index = int(random.random()*1000)
    plot_data = data[index:index+1,:,:]
    plt.figure(figsize=(10,4))
    plt.xlabel("Fchans")
    plt.ylabel("Time")
    plt.imshow(plot_data[0,:,:,0], interpolation='nearest', cmap=plt.get_cmap('hot'))
    plt.show()
  
    stuff = model.encoder.predict(plot_data)
    sample = sample_creation(stuff)
    reconstruction = model.decoder.predict(sample)
    plt.figure(figsize=(10,4))
    plt.title("reconstruction")
    plt.xlabel("Fchans")
    plt.ylabel("Time")
    plt.imshow(reconstruction[0,:,:,0], interpolation='nearest', cmap=plt.get_cmap('hot'))
